In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from collections import defaultdict

from IPython.display import HTML
from IPython.display import display_html

%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")
from utils.utils import *
import os


In [2]:
#FN = "../raw_views_filters_2021-12-15.ss.csv"
INP_FIELD = "NormInputHashCode"
FILTER_FIELD = "FilterExpr"
DEDUP_FIELDS = [INP_FIELD, FILTER_FIELD]

FN = "../data/cosmos09/raw_views_filters_2021-12-15.ss.csv"
DEDUPFN = FN.replace(".ss.csv", "-dedup.ss.csv")
#INP_FIELD = "HashTagInput"

In [3]:
#df = pd.read_csv(FN, sep="|")
#df = pd.read_csv(FN, sep="\t")
import time
start = time.time()
if os.path.exists(DEDUPFN):
    df = pd.read_csv(DEDUPFN, sep="\t")
else:
    df = pd.read_csv(FN, sep="\t")
    
print("took: ", time.time()-start)

took:  4.25859260559082


In [4]:
print(len(df))

659504


In [5]:
#df = df.rename(columns={"#Field:JobGUID":"jobid"})
df = df.rename(columns={"JobGUID":"jobid"})

In [6]:
print(df.keys())

Index(['jobid', 'HashTagInput', 'OperatorName', 'EstCardinality',
       'InputCardinality', 'NormInputHashCode', 'RowCount', 'Selectivity',
       'ExclusiveTime', 'StageName', 'VertexCount', 'FilterExpr'],
      dtype='object')


In [7]:
# For some analysis, we may not want to drop duplicates here.

## not sure why this drops so many too; same job, multiple filters on same table is possible, but why is it so common?
#df = df.drop_duplicates(["jobid", INP_FIELD])

df = df.drop_duplicates(DEDUP_FIELDS)

In [8]:
print(len(df))

659504


In [9]:
df["InputCardinality"] = df.apply(lambda x: max(1, x["InputCardinality"]), axis=1)
#df["InputCardinality"] = df.apply(lambda x: max(x["RowCount"], x["InputCardinality"]), axis=1)
df["Selectivity"] = df.apply(lambda x: min(x["RowCount"] / x["InputCardinality"], 1.0) ,axis=1)

In [10]:
#df = df.sample(frac=0.1)

In [17]:
start = time.time()

inpdata = defaultdict(list)
# filter_id, inp_id, col, op, val, card

for idx, row in df.iterrows():
    curdata = defaultdict(list)
    
    cur_row = row
    expr = row[FILTER_FIELD]
    exprhash = deterministic_hash(expr)
    inp = deterministic_hash(row[INP_FIELD])
    
    try:
        d = json.loads(expr)
    except:
        continue
    
    rowcount = row["RowCount"]
    total = row["InputCardinality"]
    
    filter_values = extract_values(d, "values")
    ops = extract_values(d, "expOperator")
    children = extract_values(d, "children")
    cols = extract_values(d, "name")
    dtypes = extract_values(d, "dataType")
    skip = False
    
    # we will ignore if it has a like
    for fi, fvs in enumerate(filter_values):
        if ops[fi] is None:
            continue
        if ops[fi] == "Or":
            continue
        if ops[fi] == "And":
            continue
        if cols[fi] == "":
            continue
        if len(fvs) == 0:
            continue
            
        if ops[fi] == "=" or ops[fi] == "!=":
            curdata["inp"].append(inp)
            curdata["expr"].append(exprhash)
            
            curdata["column"].append(deterministic_hash(cols[fi]))
            curdata["op"].append(ops[fi])

            curdata["val"].append(deterministic_hash(fvs[0]))

            curdata["rowcount"].append(rowcount)
            curdata["total"].append(total)
            curdata["dtype"].append(dtypes[fi])
        elif ">" in ops[fi] or "<" in ops[fi]:
            _, pval = get_cont_dtype(fvs[0])
            curdata["inp"].append(inp)
            curdata["expr"].append(exprhash)
            curdata["column"].append(deterministic_hash(cols[fi]))
            curdata["op"].append(ops[fi])
            curdata["val"].append(pval)

            curdata["rowcount"].append(rowcount)
            curdata["total"].append(total)
            curdata["dtype"].append(dtypes[fi])
        else:
            # skip the whole thing as we don't know what it might be
            skip = True
            break
            
    if not skip:
        for k,v in curdata.items():
            inpdata[k] += v
            
print("took: ", time.time()-start)

took:  152.6686987876892


In [18]:
len(inpdata["inp"])

1251906

In [19]:
inputdf = pd.DataFrame(inpdata)

In [20]:
print(len(set(inputdf["inp"])))
print(len(set(inputdf["expr"])))

23592
436220


In [37]:
inputdf.groupby(["inp"]).nunique()["expr"].reset_index().sort_values(by="expr", ascending=False).head(100)

,inp,expr
4187,255773240420717358254595498893360200767708373681,95687
17055,1054297722355938783602213506601570792275526208974,66850
20797,1284728961546971383548154329157411865345199210853,52524
3469,209683720027035330456987871903411900941757976299,51876
6839,420144688275135030303836302705165426502744219190,51876
9608,590098356872375275786509737619998809409148285796,13481
23450,1452315673842759648496640150048345542490118411594,12749
5700,348535770876243502078312647511546058095430536544,11763
22114,1370289182471235843135819159973683982066096969853,11431
12266,753157818166685136743036487587033878514932019771,10808


In [22]:
pd.set_option('display.max_rows', 500)
inputdf.head(100)

,inp,expr,column,op,val,rowcount,total,dtype
0,446518283181518659655284803327031818445091004072,462337974510202744814136396390643701061785295835,92717550979937297620041048120837373017599121300,!=,783537480200300836008736957935041156932762439892,71590454,1.224900e+08,String
1,348535770876243502078312647511546058095430536544,172196654126615122863816571645839210767468251998,753953770705074531072006683054711322983725468890,=,711994474182486731904495729972800452696204299012,189352,4.485550e+09,Bool
2,348535770876243502078312647511546058095430536544,172196654126615122863816571645839210767468251998,660704121368068409124109682333762412573266388513,=,711994474182486731904495729972800452696204299012,189352,4.485550e+09,Bool
3,348535770876243502078312647511546058095430536544,172196654126615122863816571645839210767468251998,119220241930998618422194734248182707360023243948,>=,53777110,189352,4.485550e+09,Int
4,1233280883302306592014300525777266519730563502981,1318360397040063118312371870658893695603483582692,435597428601583694656718752664577829045311706582,>=,1636806600.0,699938,1.454370e+10,Date
5,1233280883302306592014300525777266519730563502981,1318360397040063118312371870658893695603483582692,435597428601583694656718752664577829045311706582,<=,1636808400.0,699938,1.454370e+10,Date
6,706633419906613412947694884737697887364469527202,753584435787886469700553946629414892875271958511,1024650897683648897771065403804041231532468543037,>=,1636806600.0,38296,7.481310e+08,Date
7,706633419906613412947694884737697887364469527202,753584435787886469700553946629414892875271958511,1024650897683648897771065403804041231532468543037,<=,1636808400.0,38296,7.481310e+08,Date
8,779259894942154278022726321295679687538471920935,191984090478738225539313984709789957408587010740,1162191112903482786656125942092154300071344912929,>=,1636806600.0,2621418,1.290910e+10,Date
9,779259894942154278022726321295679687538471920935,191984090478738225539313984709789957408587010740,1162191112903482786656125942092154300071344912929,<=,1636808400.0,2621418,1.290910e+10,Date


In [23]:
inputdf.sort_values(by="rowcount", ascending=False).head(100)

,inp,expr,column,op,val,rowcount,total,dtype
574488,1135354677733482194836098280586234492866339572808,682932016312556171183245921756783839947505430812,617908181400825461237279725894180318629905254909,>,0,1365339033988,2.622020e+11,Int
550058,1132443584784904623608255028211547443860609816659,270103298814808550831880055051374501684661234650,231604068237102763164424812528453237350830220323,>=,1619827200.0,1324946942748,6.766650e+11,Date
540299,1132443584784904623608255028211547443860609816659,661029552038843374866365545357374632267616760621,231604068237102763164424812528453237350830220323,>=,1619827200.0,1324946942748,6.766650e+11,Date
540300,1132443584784904623608255028211547443860609816659,661029552038843374866365545357374632267616760621,231604068237102763164424812528453237350830220323,<=,1638230400.0,1324946942748,6.766650e+11,Date
550059,1132443584784904623608255028211547443860609816659,270103298814808550831880055051374501684661234650,231604068237102763164424812528453237350830220323,<=,1638230400.0,1324946942748,6.766650e+11,Date
724153,274036878710732123359469920321574226370030911975,417760627814001853015562548491436507059392840079,169848428514964573399013074667433797788549644913,=,304942582444936629325699363757435820077590259883,995663814402,1.061380e+12,Int
724152,274036878710732123359469920321574226370030911975,417760627814001853015562548491436507059392840079,969573923301427236987342698928831851904532978331,>,30.0,995663814402,1.061380e+12,Double
1203855,756391200497752771600551464768104665295122913883,870830155137841784170360230298029955871845066881,796752808438870941544988363060297187338771915977,!=,440292606951320038970009429333361249523593781569,947490432109,3.084470e+12,String
1244522,159158213914365058075288969182740265590307050232,617424779151787567251187556649585642097428873944,570545366109909422077253575373061919533751981842,<=,1,819744866530,7.902040e+12,Int
770493,374869627686925023774580831965849731494586338120,606474647032782376952129657002444279669828984534,1195572595773649167270050718717743319874719318555,!=,1262621932822635120280851075635352199403200521605,792771013011,1.517020e+13,String


In [24]:
inputdf.sort_values(by="rowcount", ascending=True).head(100)

,inp,expr,column,op,val,rowcount,total,dtype
584120,1015183128526043218668460855624680641489076443647,963570422326303326819865878959253791265960150063,505853073703993321121603606177751790647203612368,<,1638075600.0,0,8.440193e+10,Date
1002605,1342650135054896305716492228899195939407105878123,67624259728412242299734681777947389096357597015,806388113356579584582302194631660158803897753176,=,332216073419518953525261050934537404784031478078,0,9.830080e+08,String
1002606,296359746384585912075137051706075599270715357873,1317393981072210018566061000419830957120247003256,1038218154473304945349414268144322050900458059139,>=,1638973800.0,0,3.498700e+10,Date
1002607,296359746384585912075137051706075599270715357873,1317393981072210018566061000419830957120247003256,1038218154473304945349414268144322050900458059139,<=,1638975600.0,0,3.498700e+10,Date
1002608,296359746384585912075137051706075599270715357873,1317393981072210018566061000419830957120247003256,1179771036664437845752027422086429165891485376248,!=,1184277895496617267897092167156895951528234514932,0,3.498700e+10,String
1002609,296359746384585912075137051706075599270715357873,1317393981072210018566061000419830957120247003256,1179771036664437845752027422086429165891485376248,!=,301850938659087737044192929008639121894428757850,0,3.498700e+10,String
1002610,296359746384585912075137051706075599270715357873,1317393981072210018566061000419830957120247003256,1179771036664437845752027422086429165891485376248,!=,477158217771132476692309778434423845558676883125,0,3.498700e+10,String
1002611,296359746384585912075137051706075599270715357873,1317393981072210018566061000419830957120247003256,1179771036664437845752027422086429165891485376248,!=,332216073419518953525261050934537404784031478078,0,3.498700e+10,String
787238,11333431479726943123790055320932465968573932870,577594174102539350377820260295019412391705199051,288784615527963252460330284820813376419490128633,!=,1171293193490232083748057528168371172317157005446,0,1.770830e+11,String
787237,11333431479726943123790055320932465968573932870,577594174102539350377820260295019412391705199051,288784615527963252460330284820813376419490128633,!=,553129369994634877067667934518617876376376905444,0,1.770830e+11,String


In [26]:
INPUT_DATA_FN = FN.replace(".ss.csv", "-input-data.csv")

In [27]:
inputdf.to_csv(INPUT_DATA_FN, index=False)

In [28]:
inputdf2 = pd.read_csv(INPUT_DATA_FN)

/home/pari/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [29]:
inputdf2.head(10)

,inp,expr,column,op,val,rowcount,total,dtype
0,446518283181518659655284803327031818445091004072,462337974510202744814136396390643701061785295835,92717550979937297620041048120837373017599121300,!=,783537480200300836008736957935041156932762439892,71590454,1.224900e+08,String
1,348535770876243502078312647511546058095430536544,172196654126615122863816571645839210767468251998,753953770705074531072006683054711322983725468890,=,711994474182486731904495729972800452696204299012,189352,4.485550e+09,Bool
2,348535770876243502078312647511546058095430536544,172196654126615122863816571645839210767468251998,660704121368068409124109682333762412573266388513,=,711994474182486731904495729972800452696204299012,189352,4.485550e+09,Bool
3,348535770876243502078312647511546058095430536544,172196654126615122863816571645839210767468251998,119220241930998618422194734248182707360023243948,>=,53777110,189352,4.485550e+09,Int
4,1233280883302306592014300525777266519730563502981,1318360397040063118312371870658893695603483582692,435597428601583694656718752664577829045311706582,>=,1636806600.0,699938,1.454370e+10,Date
5,1233280883302306592014300525777266519730563502981,1318360397040063118312371870658893695603483582692,435597428601583694656718752664577829045311706582,<=,1636808400.0,699938,1.454370e+10,Date
6,706633419906613412947694884737697887364469527202,753584435787886469700553946629414892875271958511,1024650897683648897771065403804041231532468543037,>=,1636806600.0,38296,7.481310e+08,Date
7,706633419906613412947694884737697887364469527202,753584435787886469700553946629414892875271958511,1024650897683648897771065403804041231532468543037,<=,1636808400.0,38296,7.481310e+08,Date
8,779259894942154278022726321295679687538471920935,191984090478738225539313984709789957408587010740,1162191112903482786656125942092154300071344912929,>=,1636806600.0,2621418,1.290910e+10,Date
9,779259894942154278022726321295679687538471920935,191984090478738225539313984709789957408587010740,1162191112903482786656125942092154300071344912929,<=,1636808400.0,2621418,1.290910e+10,Date


In [30]:
inputdf2.keys()

Index(['inp', 'expr', 'column', 'op', 'val', 'rowcount', 'total', 'dtype'], dtype='object')